### 参考https://github.com/lyupin/Visualize-DXY-2019-nCov-Data

### 将 BlankerL 提供的2019新型冠状病毒疫情时间序列数据仓库进行可视化。

### 可提供市级的确诊数

# 定义必要的函数

In [18]:
import pandas as pd
import numpy as np
from datetime import timedelta, date
#import seaborn as sns
import os

In [19]:
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rcParams['font.family'] = 'sans-serif'
mpl.rcParams['font.sans-serif'] = ['SimHei'] # 'Arial'

mpl.rcParams['xtick.labelsize'] = 12
mpl.rcParams['ytick.labelsize'] = 12
mpl.rcParams['legend.fontsize'] = 12
mpl.rcParams['axes.labelsize'] = 12

mpl.rcParams['xtick.direction'] = 'in'
mpl.rcParams['ytick.direction'] = 'in'

In [20]:
# 计算到某天的累积量
def select_record_until_certain_time_in_one_day(times_in, hour_target, minute_target):
    deltas = [[egg[0], timedelta(hours = hour_target - egg[1], minutes = minute_target - egg[2]).seconds] for egg in times_in]
    deltas = np.array(deltas)

    i2_min_tmp = -1
    delta_min = 99999999
    for i2 in range(deltas.shape[0]):
        delta_tmp = deltas[i2, :]
        if delta_tmp[1] > 0 and delta_tmp[1] < delta_min:
            i2_min_tmp = i2
            delta_min = delta_tmp[1]
    
    return deltas[i2_min_tmp, 0]

In [21]:
# 计算每天的累积量
def select_record_until_certain_time_on_each_day(dat_in, hour_target = 12, minute_target = 0, key = 'updateTime'):
    indices_tmp = []
    
    datetime_old = pd.to_datetime(dat_in.iat[0, 6])
    day_old = datetime_old.day
    
    hour_old = datetime_old.hour
    minute_old = datetime_old.minute
    times_in_day = [[0, hour_old, minute_old]]
    
    print([day_old, hour_old, minute_old])    
    
    for i in range(1, dat_in.shape[0]):
        datetime_tmp = pd.to_datetime(dat_in.iat[i, 6])
        print(datetime_tmp)
        
        day_tmp, hour_tmp, minute_tmp = datetime_tmp.day, datetime_tmp.hour, datetime_tmp.minute
        
        if day_tmp == day_old:
            print('Append')
            times_in_day.append([i, hour_tmp, minute_tmp])        
        else:
            print('Select')
            index_tmp = select_record_until_certain_time_in_one_day(times_in_day, hour_target, minute_target)            
            indices_tmp.append(index_tmp)
            
            day_old = day_tmp
            times_in_day = [[i, hour_tmp, minute_tmp]]
            
    print('Finalize')
    index_tmp = select_record_until_certain_time_in_one_day(times_in_day, hour_target, minute_target)            
    indices_tmp.append(index_tmp)
    
    return dat_in.iloc[indices_tmp, :]

In [22]:
# 筛选城市
def extract_by_city(df_in, cityName, calc_increment = True, print = False):
    df_1 = df_in[df_in['cityName'] == cityName]

    df_2 = df_1.iloc[::-1, [0,1,6,7,8,9,10]]
    #print(df_2.columns)
    #print(df_2)

    df_3 = select_record_until_certain_time_on_each_day(df_2)
    #print(df_3.columns)
    #print(df_3)

    datetimes = pd.to_datetime(df_3.loc[:, 'updateTime'])

    dates = [row[1].date() for row in datetimes.items()]

    df_4 = df_3.rename(columns={'updateTime': 'updateDate'})
    df_4.loc[:, 'updateDate'] = dates
    
    if calc_increment:
        df_4['confirmedIncrement'] = df_4['city_confirmedCount']
        df_4.iloc[0, 7] = 0
        if df_4.shape[0] > 1:
            for i_row in range(1, df_4.shape[0]):
                df_4.iloc[i_row, 7] = df_4.iloc[i_row, 2] - df_4.iloc[i_row - 1, 2]
                
    if print:
        print(df_4)
    
    return df_4

In [23]:
# 筛选省份
def extract_by_province(df_in, provinceName, calc_increment = True):
    df_1 = df_in[df_in['provinceName'] == provinceName]
    #print(df_1)

    #print(df_1['cityName'])
    #print(df_1['cityName'].unique())
    cityNames = df_1['cityName'].unique()
    nCity = len(cityNames)
    datasetsInProvince = []
    for i in range(nCity):
        datasetsInProvince.append(extract_by_city(df_in, cityNames[i]))
    dfInProvince = pd.concat(datasetsInProvince)
    #print(dfInProvince)

    return dfInProvince

In [24]:
# 绘制市级疫情图
def plot_city_dataset_by_matplot(df_in, provinceName, cityName, show = False, savefig = True):
    
    path_figure_cities = os.path.join(os.getcwd(), 'figures')
    if (not os.path.exists(path_figure_cities)):
        os.mkdir(path_figure_cities)

    df_tmp = df_in[df_in['cityName'] == cityName]

    # matplotlib date format object
    hfmt = mpl.dates.DateFormatter('%m/%d')

    fig, ax = plt.subplots()
    ax.plot(df_tmp['updateDate'], df_tmp['city_confirmedCount'], '.-', label = cityName)

    ax.xaxis.set_major_formatter(hfmt)
    plt.legend()
    #plt.xlabel('日期')
    plt.ylabel('累计确诊数')
    ylims = ax.get_ylim()
    ax.set_ylim([0, ylims[1]])
    if savefig:
        fn_tmp = os.path.join(path_figure_cities, \
            '累计确诊数-{:s}-{:s}.png'.format(provinceName, cityName))
        plt.savefig(fn_tmp, dpi = 100)
    if show:
        plt.show()
    plt.close()

    fig, ax = plt.subplots()
    ax.plot(df_tmp['updateDate'], df_tmp['confirmedIncrement'], '.-', label = cityName)

    ax.xaxis.set_major_formatter(hfmt)
    plt.legend()
    plt.ylabel('单日新增确诊数')
    ylims = ax.get_ylim()
    ax.set_ylim([0, ylims[1]])
    if savefig:
        fn_tmp = os.path.join(path_figure_cities,\
            '单日新增确诊数-{:s}-{:s}.png'.format(provinceName, cityName))
        plt.savefig(fn_tmp, dpi = 100)
    if show:
        plt.show()
    plt.close()

    return

In [25]:
# 绘制省级疫情图
def plot_province_dataframe_by_matplot(df_in, provinceName, show = False, savefig = True):
    
    path_figure_cities = os.path.join(os.getcwd(), 'figures')
    if (not os.path.exists(path_figure_cities)):
        os.mkdir(path_figure_cities)

    # matplotlib date format object
    hfmt = mpl.dates.DateFormatter('%m/%d')

    fig, ax = plt.subplots()
    ax.plot(df_in['updateDate'], df_in['confirmedCount'], '.-', label = provinceName)

    ax.xaxis.set_major_formatter(hfmt)
    plt.legend()
    #plt.xlabel('日期')
    plt.ylabel('累计确诊数')
    ylims = ax.get_ylim()
    ax.set_ylim([0, ylims[1]])
    if savefig:
        fn_tmp = '累计确诊数-{:s}.png'.format(provinceName)
        plt.savefig(fn_tmp, dpi = 300)
    if show:
        plt.show()
    plt.close()

    fig, ax = plt.subplots()
    ax.plot(df_in['updateDate'], df_in['confirmedIncrement'], '.-', label = provinceName)

    ax.xaxis.set_major_formatter(hfmt)
    plt.legend()
    plt.ylabel('单日新增确诊数')
    ylims = ax.get_ylim()
    ax.set_ylim([0, ylims[1]])
    if savefig:
        fn_tmp = '单日新增确诊数-{:s}.png'.format(provinceName)
        plt.savefig(fn_tmp, dpi = 300)
    if show:
        plt.show()
    plt.close()

    return

In [26]:
# 绘制趋势图
def plot_datasets_by_matplot(dat_in, cityNames, show = False, savefig = True):
    nDat = len(dat_in)
    
    # matplotlib date format object
    hfmt = mpl.dates.DateFormatter('%m/%d')

    fig, ax = plt.subplots()
    for i in range(nDat):
        df_tmp = dat_in[i]
        ax.plot(df_tmp['updateDate'], df_tmp['city_confirmedCount'], '.-', label = cityNames[i])

    ax.xaxis.set_major_formatter(hfmt)
    plt.legend()
    #plt.xlabel('日期')
    plt.ylabel('累计确诊数')
    ylims = ax.get_ylim()
    ax.set_ylim([0, ylims[1]])
    if savefig:
        plt.savefig('累计确诊数.png', dpi = 300)
    if show:
        plt.show()
    plt.close()

    fig, ax = plt.subplots()
    for i in range(nDat):
        df_tmp = dat_in[i]
        ax.plot(df_tmp['updateDate'], df_tmp['confirmedIncrement'], '.-', label = cityNames[i])

    ax.xaxis.set_major_formatter(hfmt)
    plt.legend()
    plt.ylabel('单日新增确诊数')
    ylims = ax.get_ylim()
    ax.set_ylim([0, ylims[1]])
    if savefig:
        plt.savefig('单日新增确诊数.png', dpi = 300)
    if show:
        plt.show()
    plt.close()
    
    if savefig:
        print('已保存趋势图至本目录：\n1.累计确诊数.png\n2.单日新增确诊数.png')

    return

In [27]:
# 城市趋势
def visualize_area_data_by_city(fn_csv, cityNames):
    df_orig = pd.read_csv(fn_csv)

    nCity = len(cityNames)

    datasets = []
    for i in range(nCity):
        df_tmp = extract_by_city(df_orig, cityNames[i])
        datasets.append(df_tmp)

    plot_datasets_by_matplot(datasets, cityNames)
    return

In [28]:
# 省级热力图
def visualize_area_data_by_province(fn_csv, provinceNames=None):
    df_orig = pd.read_csv(fn_csv)
    #print(df_orig.columns)

    if provinceNames is None:
        provinceNames = df_orig['provinceName'].unique()
    nProvince = len(provinceNames)

    datasets = []
    for i in range(nProvince):
        df_tmp = extract_by_province(df_orig, provinceNames[i])
        datasets.append(df_tmp)

        cityNames = df_tmp['cityName'].unique()
        nCity = len(cityNames)

        for iCity in range(nCity):
            print('Plotting {:s} {:s}'.format(provinceNames[i],
                                              cityNames[iCity]))
            plot_city_dataset_by_matplot(df_tmp, provinceNames[i],
                                         cityNames[iCity])

        dates = df_tmp['updateDate'].unique()
        nDate = len(dates)
        confirmedSumInProvince = np.zeros([nDate], dtype='int')
        incrementSumInProvince = np.zeros([nDate], dtype='int')

        for iDate in range(nDate):
            dfInDay_tmp = df_tmp[df_tmp['updateDate'] == dates[iDate]]
            confirmedSumInProvince[iDate] = dfInDay_tmp[
                'city_confirmedCount'].sum()
            incrementSumInProvince[iDate] = dfInDay_tmp[
                'confirmedIncrement'].sum()

        dfProvince_tmp = pd.DataFrame({'provinceName': provinceNames[i], \
         'confirmedCount': confirmedSumInProvince, \
         'confirmedIncrement': incrementSumInProvince, 'updateDate': dates})
        dfProvince_tmp = dfProvince_tmp.sort_values(by='updateDate')
        #print(dfProvince_tmp)
        plot_province_dataframe_by_matplot(dfProvince_tmp, provinceNames[i])
    return

# 调用函数进行可视化

In [13]:
# 指定 'DXYArea.csv'文件的路径

fn_DXYArea = 'C:/Users/123/Desktop/2019_nCov/data/csv_github/DXYArea.csv'

In [15]:
# 绘制城市趋势图
cityNames = ['合肥', '南京', '扬州']
visualize_area_data_by_city(fn_DXYArea, cityNames)

[24, 3, 50]
2020-01-24 09:25:27.267000
Append
2020-01-24 10:13:53.313000
Append
2020-01-24 10:14:53.875000
Append
2020-01-24 10:15:54.468000
Append
2020-01-24 10:16:55.029000
Append
2020-01-24 23:35:03.158000
Append
2020-01-25 09:52:46.972000
Select
2020-01-25 10:58:24.313000
Append
2020-01-25 10:59:24.817000
Append
2020-01-25 11:00:25.298000
Append
2020-01-25 11:01:25.954000
Append
2020-01-26 08:40:07.920000
Select
2020-01-26 09:53:44.753000
Append
2020-01-27 08:47:51.117000
Select
2020-01-28 09:04:57.271000
Select
2020-01-29 08:40:14.976000
Select
2020-01-29 12:39:59.492000
Append
2020-01-29 12:47:02.222000
Append
2020-01-29 12:48:02.638000
Append
2020-01-29 13:37:23.573000
Append
2020-01-29 13:42:25.933000
Append
2020-01-30 08:55:06.671000
Select
2020-01-31 08:39:44.417000
Select
2020-02-01 09:16:36.004000
Select
2020-02-01 09:17:37.792000
Append
2020-02-01 09:18:39.640000
Append
2020-02-01 09:19:41.759000
Append
2020-02-01 09:24:50.820000
Append
2020-02-01 16:09:04.135000
Append
20

In [16]:
# 绘制省份趋势图
provinceNames = ['安徽省']
visualize_area_data_by_province(fn_DXYArea, provinceNames)


[24, 3, 50]
2020-01-24 09:25:27.267000
Append
2020-01-24 10:13:53.313000
Append
2020-01-24 10:14:53.875000
Append
2020-01-24 10:15:54.468000
Append
2020-01-24 10:16:55.029000
Append
2020-01-24 23:35:03.158000
Append
2020-01-25 09:52:46.972000
Select
2020-01-25 10:58:24.313000
Append
2020-01-25 10:59:24.817000
Append
2020-01-25 11:00:25.298000
Append
2020-01-25 11:01:25.954000
Append
2020-01-26 08:40:07.920000
Select
2020-01-26 09:53:44.753000
Append
2020-01-27 08:47:51.117000
Select
2020-01-28 09:04:57.271000
Select
2020-01-29 08:40:14.976000
Select
2020-01-29 12:39:59.492000
Append
2020-01-29 12:47:02.222000
Append
2020-01-29 12:48:02.638000
Append
2020-01-29 13:37:23.573000
Append
2020-01-29 13:42:25.933000
Append
2020-01-30 08:55:06.671000
Select
2020-01-31 08:39:44.417000
Select
2020-02-01 09:16:36.004000
Select
2020-02-01 09:17:37.792000
Append
2020-02-01 09:18:39.640000
Append
2020-02-01 09:19:41.759000
Append
2020-02-01 09:24:50.820000
Append
2020-02-01 16:09:04.135000
Append
20

[24, 3, 50]
2020-01-24 09:25:27.267000
Append
2020-01-24 10:13:53.313000
Append
2020-01-24 10:14:53.875000
Append
2020-01-24 10:15:54.468000
Append
2020-01-24 10:16:55.029000
Append
2020-01-24 23:35:03.158000
Append
2020-01-25 09:52:46.972000
Select
2020-01-25 10:58:24.313000
Append
2020-01-25 10:59:24.817000
Append
2020-01-25 11:00:25.298000
Append
2020-01-25 11:01:25.954000
Append
2020-01-26 08:40:07.920000
Select
2020-01-26 09:53:44.753000
Append
2020-01-27 08:47:51.117000
Select
2020-01-28 09:04:57.271000
Select
2020-01-29 08:40:14.976000
Select
2020-01-29 12:39:59.492000
Append
2020-01-29 12:47:02.222000
Append
2020-01-29 12:48:02.638000
Append
2020-01-29 13:37:23.573000
Append
2020-01-29 13:42:25.933000
Append
2020-01-30 08:55:06.671000
Select
2020-01-31 08:39:44.417000
Select
2020-02-01 09:16:36.004000
Select
2020-02-01 09:17:37.792000
Append
2020-02-01 09:18:39.640000
Append
2020-02-01 09:19:41.759000
Append
2020-02-01 09:24:50.820000
Append
2020-02-01 16:09:04.135000
Append
20

[25, 10, 59]
2020-01-25 11:00:25.298000
Append
2020-01-25 11:01:25.954000
Append
2020-01-26 08:40:07.920000
Select
2020-01-26 09:53:44.753000
Append
2020-01-27 08:47:51.117000
Select
2020-01-28 09:04:57.271000
Select
2020-01-29 08:40:14.976000
Select
2020-01-29 12:39:59.492000
Append
2020-01-29 12:47:02.222000
Append
2020-01-29 12:48:02.638000
Append
2020-01-29 13:37:23.573000
Append
2020-01-29 13:42:25.933000
Append
2020-01-30 08:55:06.671000
Select
2020-01-31 08:39:44.417000
Select
2020-02-01 09:16:36.004000
Select
2020-02-01 09:17:37.792000
Append
2020-02-01 09:18:39.640000
Append
2020-02-01 09:19:41.759000
Append
2020-02-01 09:24:50.820000
Append
2020-02-01 16:09:04.135000
Append
2020-02-01 18:29:15.389000
Append
2020-02-01 19:25:27.786000
Append
2020-02-01 21:30:09.626000
Append
2020-02-02 09:29:19.481000
Select
2020-02-02 18:32:12.770000
Append
2020-02-02 18:33:19.385000
Append
2020-02-02 21:04:21.465000
Append
2020-02-03 09:28:34.053000
Select
2020-02-03 12:22:36.311000
Append
2

2020-02-08 13:08:26.921000
Append
2020-02-08 13:30:45.076000
Append
2020-02-08 15:36:22.375000
Append
2020-02-09 09:19:05.325000
Select
2020-02-09 13:23:31.311000
Append
2020-02-09 13:24:33.164000
Append
2020-02-09 13:25:33.972000
Append
2020-02-09 14:18:21.144000
Append
2020-02-09 16:35:16.423000
Append
2020-02-09 17:22:57.435000
Append
2020-02-09 17:23:58.288000
Append
Finalize
[26, 9, 53]
2020-01-27 08:47:51.117000
Select
2020-01-28 09:04:57.271000
Select
2020-01-29 08:40:14.976000
Select
2020-01-29 12:39:59.492000
Append
2020-01-29 12:47:02.222000
Append
2020-01-29 12:48:02.638000
Append
2020-01-29 13:37:23.573000
Append
2020-01-29 13:42:25.933000
Append
2020-01-30 08:55:06.671000
Select
2020-01-31 08:39:44.417000
Select
2020-02-01 09:16:36.004000
Select
2020-02-01 09:17:37.792000
Append
2020-02-01 09:18:39.640000
Append
2020-02-01 09:19:41.759000
Append
2020-02-01 09:24:50.820000
Append
2020-02-01 16:09:04.135000
Append
2020-02-01 18:29:15.389000
Append
2020-02-01 19:25:27.786000


In [ ]:
# 绘制所有省份的趋势图
visualize_area_data_by_province(fn_DXYArea)

[24, 3, 50]
2020-01-24 10:34:04.301000
Append
2020-01-24 10:35:04.903000
Append
2020-01-24 23:35:03.158000
Append
2020-01-25 12:32:21.218000
Select
2020-01-25 12:44:28.816000
Append
2020-01-25 12:45:29.371000
Append
2020-01-25 20:07:11.173000
Append
2020-01-25 20:08:11.787000
Append
2020-01-26 11:22:01.594000
Select
2020-01-27 12:41:59.541000
Select
2020-01-27 18:25:44.242000
Append
2020-01-28 09:50:34.189000
Select
2020-01-28 15:43:29.923000
Append
2020-01-29 09:33:38.618000
Select
2020-01-29 10:15:58.978000
Append
2020-01-30 12:50:59.565000
Select
2020-01-30 13:25:14.824000
Append
2020-01-31 08:04:10.771000
Select
2020-01-31 09:53:12.740000
Append
2020-01-31 13:06:41.642000
Append
2020-01-31 13:44:26.419000
Append
2020-01-31 13:45:33.073000
Append
2020-01-31 13:46:39.815000
Append
2020-01-31 17:03:38.927000
Append
2020-02-01 07:47:50.494000
Select
2020-02-01 10:13:34.702000
Append
2020-02-01 11:49:31.363000
Append
2020-02-01 12:00:59.663000
Append
2020-02-01 13:21:04.249000
Append
20

[25, 12, 44]
2020-01-25 12:45:29.371000
Append
2020-01-25 20:07:11.173000
Append
2020-01-25 20:08:11.787000
Append
2020-01-26 11:22:01.594000
Select
2020-01-27 12:41:59.541000
Select
2020-01-27 18:25:44.242000
Append
2020-01-28 09:50:34.189000
Select
2020-01-28 15:43:29.923000
Append
2020-01-29 09:33:38.618000
Select
2020-01-29 10:15:58.978000
Append
2020-01-30 12:50:59.565000
Select
2020-01-30 13:25:14.824000
Append
2020-01-31 08:04:10.771000
Select
2020-01-31 09:53:12.740000
Append
2020-01-31 13:06:41.642000
Append
2020-01-31 13:44:26.419000
Append
2020-01-31 13:45:33.073000
Append
2020-01-31 13:46:39.815000
Append
2020-01-31 17:03:38.927000
Append
2020-02-01 07:47:50.494000
Select
2020-02-01 10:13:34.702000
Append
2020-02-01 11:49:31.363000
Append
2020-02-01 12:00:59.663000
Append
2020-02-01 13:21:04.249000
Append
2020-02-01 16:09:04.135000
Append
2020-02-01 16:28:53.944000
Append
2020-02-01 16:44:57.532000
Append
2020-02-01 18:29:15.389000
Append
2020-02-01 20:56:15.182000
Append
2

2020-02-05 10:52:10.634000
Append
2020-02-05 19:11:21.350000
Append
2020-02-05 19:12:22.091000
Append
2020-02-06 00:43:03.849000
Select
2020-02-06 10:21:55.615000
Append
2020-02-06 13:19:08.957000
Append
2020-02-06 13:22:11.260000
Append
2020-02-06 19:06:30.391000
Append
2020-02-06 19:14:35.927000
Append
2020-02-07 08:01:59.048000
Select
2020-02-07 08:04:00.779000
Append
2020-02-07 09:55:21.805000
Append
2020-02-07 09:56:22.507000
Append
2020-02-07 10:00:25.856000
Append
2020-02-07 10:02:27.566000
Append
2020-02-07 10:20:41.420000
Append
2020-02-07 10:36:52.697000
Append
2020-02-07 10:38:54.379000
Append
2020-02-07 10:39:55.095000
Append
2020-02-07 13:41:15.423000
Append
2020-02-07 18:38:10.062000
Append
2020-02-07 20:32:40.930000
Append
2020-02-08 10:29:21.794000
Select
2020-02-08 18:18:36.711000
Append
2020-02-08 18:19:37.487000
Append
2020-02-09 09:46:28.135000
Select
2020-02-09 10:00:40.877000
Append
2020-02-09 16:51:31.295000
Append
2020-02-09 18:18:47.539000
Append
2020-02-09 19:

Append
2020-01-31 13:46:39.815000
Append
2020-01-31 17:03:38.927000
Append
2020-02-01 07:47:50.494000
Select
2020-02-01 10:13:34.702000
Append
2020-02-01 11:49:31.363000
Append
2020-02-01 12:00:59.663000
Append
2020-02-01 13:21:04.249000
Append
2020-02-01 16:09:04.135000
Append
2020-02-01 16:28:53.944000
Append
2020-02-01 16:44:57.532000
Append
2020-02-01 18:29:15.389000
Append
2020-02-01 20:56:15.182000
Append
2020-02-02 08:55:23.660000
Select
2020-02-02 10:01:08.780000
Append
2020-02-02 14:51:24.508000
Append
2020-02-02 17:03:13.538000
Append
2020-02-02 17:12:00.674000
Append
2020-02-02 17:47:18.345000
Append
2020-02-02 17:48:24.547000
Append
2020-02-02 22:53:58.118000
Append
2020-02-02 23:47:32.964000
Append
2020-02-03 09:28:34.053000
Select
2020-02-03 10:32:24.434000
Append
2020-02-03 14:53:22.173000
Append
2020-02-03 22:12:26.177000
Append
2020-02-03 22:13:31.840000
Append
2020-02-04 08:34:36.325000
Select
2020-02-04 13:00:36.172000
Append
2020-02-04 19:26:47.312000
Append
2020-02

Append
2020-02-05 09:19:14.153000
Select
2020-02-05 13:40:15.806000
Append
2020-02-05 13:41:25.194000
Append
2020-02-05 13:42:32.850000
Append
2020-02-06 09:44:28.627000
Select
2020-02-06 09:47:31.251000
Append
2020-02-06 11:16:37.302000
Append
2020-02-06 11:17:37.863000
Append
2020-02-06 11:21:40.576000
Append
2020-02-06 11:37:51.994000
Append
2020-02-06 11:55:05.586000
Append
2020-02-06 12:26:29.302000
Append
2020-02-06 14:26:59.208000
Append
2020-02-06 14:27:59.710000
Append
2020-02-06 15:20:38.264000
Append
2020-02-06 17:36:17.449000
Append
2020-02-06 19:17:38.469000
Append
2020-02-06 19:18:39.210000
Append
2020-02-07 09:22:56.451000
Select
2020-02-07 09:23:57.243000
Append
2020-02-07 09:26:59.553000
Append
2020-02-07 09:31:03.025000
Append
2020-02-07 11:17:25.967000
Append
2020-02-07 11:17:25.967000
Append
2020-02-07 11:18:26.758000
Append
2020-02-07 11:19:27.770000
Append
2020-02-07 11:20:28.555000
Append
2020-02-07 11:23:30.847000
Append
2020-02-07 13:29:06.431000
Append
2020-02

...

[31, 7, 33]
2020-01-31 13:44:26.419000
Append
2020-01-31 13:45:33.073000
Append
2020-01-31 13:46:39.815000
Append
2020-01-31 13:48:53.003000
Append
2020-01-31 14:06:38.496000
Append
2020-01-31 14:09:58.572000
Append
2020-01-31 14:11:05.398000
Append
2020-01-31 15:47:53.860000
Append
2020-01-31 15:49:03.700000
Append
2020-02-01 07:36:29.700000
Select
2020-02-01 07:37:31.719000
Append
2020-02-01 07:38:33.465000
Append
2020-02-01 07:48:52.295000
Append
2020-02-01 09:42:27.444000
Append
2020-02-01 16:09:04.135000
Append
2020-02-01 16:12:10.896000
Append
2020-02-01 16:13:13.054000
Append
2020-02-01 16:15:22.857000
Append
2020-02-01 18:29:15.389000
Append
2020-02-01 19:16:02.527000
Append
2020-02-02 07:36:33.155000
Select
2020-02-02 16:18:43.725000
Append
2020-02-02 16:23:09.066000
Append
2020-02-03 09:28:34.053000
Select
2020-02-03 16:26:14.278000
Append
2020-02-03 16:28:24.902000
Append
2020-02-03 16:29:32.407000
Append
2020-02-04 07:38:14.462000
Select
2020-02-04 16:14:49.980000
Append
20

Append
2020-02-03 16:28:24.902000
Append
2020-02-03 16:29:32.407000
Append
2020-02-04 07:38:14.462000
Select
2020-02-04 16:14:49.980000
Append
2020-02-04 16:15:55.937000
Append
2020-02-04 20:25:40.282000
Append
2020-02-05 07:36:51.761000
Select
2020-02-05 07:55:19.362000
Append
2020-02-05 08:01:50.496000
Append
2020-02-05 08:06:12.145000
Append
2020-02-05 08:13:48.577000
Append
2020-02-05 17:12:58.032000
Append
2020-02-05 17:13:58.583000
Append
2020-02-05 17:14:59.139000
Append
2020-02-06 07:33:50.220000
Select
2020-02-06 16:34:31.583000
Append
2020-02-06 16:35:32.320000
Append
2020-02-06 16:36:33.159000
Append
2020-02-06 18:40:08.374000
Append
2020-02-06 21:37:25.233000
Append
2020-02-06 21:38:25.888000
Append
2020-02-06 21:39:26.688000
Append
2020-02-07 07:32:39.985000
Select
2020-02-07 07:40:44.667000
Append
2020-02-07 17:12:00.499000
Append
2020-02-07 17:13:01.055000
Append
2020-02-07 17:31:15.795000
Append
2020-02-07 19:30:50.359000
Append
2020-02-07 19:31:51.163000
Append
2020-02

[27, 9, 16]
2020-01-27 09:19:08.121000
Append
2020-01-27 09:20:08.658000
Append
2020-01-28 09:15:03.842000
Select
2020-01-29 09:26:35.296000
Select
2020-01-29 10:25:03.418000
Append
2020-01-29 10:53:16.343000
Append
2020-01-30 07:08:20.707000
Select
2020-01-30 16:21:38.200000
Append
2020-01-30 16:23:39.230000
Append
2020-01-30 16:24:39.855000
Append
2020-01-31 07:33:07.210000
Select
2020-01-31 13:44:26.419000
Append
2020-01-31 13:45:33.073000
Append
2020-01-31 13:46:39.815000
Append
2020-01-31 13:48:53.003000
Append
2020-01-31 14:06:38.496000
Append
2020-01-31 14:09:58.572000
Append
2020-01-31 14:11:05.398000
Append
2020-01-31 15:47:53.860000
Append
2020-01-31 15:49:03.700000
Append
2020-02-01 07:36:29.700000
Select
2020-02-01 07:37:31.719000
Append
2020-02-01 07:38:33.465000
Append
2020-02-01 07:48:52.295000
Append
2020-02-01 09:42:27.444000
Append
2020-02-01 16:09:04.135000
Append
2020-02-01 16:12:10.896000
Append
2020-02-01 16:13:13.054000
Append
2020-02-01 16:15:22.857000
Append
20

[4, 7, 38]
2020-02-04 16:14:49.980000
Append
2020-02-04 16:15:55.937000
Append
2020-02-04 20:25:40.282000
Append
2020-02-05 07:36:51.761000
Select
2020-02-05 07:55:19.362000
Append
2020-02-05 08:01:50.496000
Append
2020-02-05 08:06:12.145000
Append
2020-02-05 08:13:48.577000
Append
2020-02-05 17:12:58.032000
Append
2020-02-05 17:13:58.583000
Append
2020-02-05 17:14:59.139000
Append
2020-02-06 07:33:50.220000
Select
2020-02-06 16:34:31.583000
Append
2020-02-06 16:35:32.320000
Append
2020-02-06 16:36:33.159000
Append
2020-02-06 18:40:08.374000
Append
2020-02-06 21:37:25.233000
Append
2020-02-06 21:38:25.888000
Append
2020-02-06 21:39:26.688000
Append
2020-02-07 07:32:39.985000
Select
2020-02-07 07:40:44.667000
Append
2020-02-07 17:12:00.499000
Append
2020-02-07 17:13:01.055000
Append
2020-02-07 17:31:15.795000
Append
2020-02-07 19:30:50.359000
Append
2020-02-07 19:31:51.163000
Append
2020-02-07 20:01:13.734000
Append
2020-02-07 20:02:14.523000
Append
2020-02-08 07:27:51.619000
Select
202

[31, 13, 46]
2020-01-31 13:48:53.003000
Append
2020-01-31 14:06:38.496000
Append
2020-01-31 14:09:58.572000
Append
2020-01-31 14:11:05.398000
Append
Finalize
[31, 13, 48]
2020-01-31 14:06:38.496000
Append
2020-01-31 14:09:58.572000
Append
2020-01-31 14:11:05.398000
Append
Finalize
[31, 13, 48]
2020-01-31 14:06:38.496000
Append
2020-01-31 14:09:58.572000
Append
2020-01-31 14:11:05.398000
Append
Finalize
[31, 13, 48]
2020-01-31 14:06:38.496000
Append
2020-01-31 14:09:58.572000
Append
Finalize
[31, 13, 45]
2020-01-31 13:46:39.815000
Append
2020-01-31 13:48:53.003000
Append
2020-01-31 14:06:38.496000
Append
2020-01-31 14:09:58.572000
Append
Finalize
[31, 13, 48]
2020-01-31 14:06:38.496000
Append
2020-01-31 14:09:58.572000
Append
Finalize
[31, 13, 44]
2020-01-31 13:45:33.073000
Append
2020-01-31 13:46:39.815000
Append
2020-01-31 13:48:53.003000
Append
2020-01-31 14:06:38.496000
Append
Finalize
[31, 13, 48]
Finalize
[31, 13, 48]
Finalize
Plotting 重庆市 万州区
Plotting 重庆市 江北区
Plotting 重庆市 云阳县
Plo